In [1]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from trl import SFTTrainer
import json, random


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/anaconda3/envs/PyEnv/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/anaconda3/envs/PyEnv/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/opt/anaconda3/envs/PyEnv/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/envs/PyEnv/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
 

In [3]:
# ✅ STEP 7: Load Tokenizer and Model (TinyLLaMA or Chat Model)
model_id = "TinyLlama/TinyLlama_v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Create the BitsAndBytesConfig object
bnb_config = BitsAndBytesConfig(load_in_8bit=True)  # For 8-bit quantization

# Load the model with quantization_config
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=bnb_config)

ImportError: Using `bitsandbytes` 8-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [ ]:
# 📤 STEP 8: Reload JSONL as HuggingFace Dataset
dataset = load_dataset("json", data_files="cot_cbt_dataset.jsonl")["train"]

In [ ]:
# ⚙️ STEP 9: Define TrainingArguments
training_args = TrainingArguments(
    output_dir="./cbt_cot_model",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    save_steps=200,
    logging_steps=20,
    learning_rate=2e-5,
    fp16=True,
)

In [ ]:
# 🚀 STEP 10: Fine-Tune Using SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=training_args,
)

trainer.train()
trainer.save_model()

In [ ]:
# 🤖 STEP 11: Inference
from transformers import pipeline

pipe = pipeline("text-generation", model="./cbt_cot_model", tokenizer=tokenizer)

prompt = tokenizer.apply_chat_template([
    {"role": "user", "content": "I can’t stop overthinking at night and it’s ruining my sleep. What should I do?"}
], tokenize=False, add_generation_prompt=True)

response = pipe(prompt, max_new_tokens=400)[0]['generated_text']
print(response)